In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp').getOrCreate()

In [3]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True, sep='\t')

In [4]:
data.show()

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if thats th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [ ]:
data.show()

In [6]:
from pyspark.sql.functions import length

In [7]:
data = data.withColumn('length', length(data['text']))

In [ ]:
data.show()

In [ ]:
data.groupBy('class').mean().show()

In [8]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [9]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vect = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [10]:
from pyspark.ml.feature import VectorAssembler

In [11]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol = 'features')

In [13]:
from pyspark.ml.classification import NaiveBayes

In [14]:
nb = NaiveBayes()

In [12]:
from pyspark.ml import Pipeline

In [15]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vect, idf, clean_up])

In [16]:
cleaner = data_prep_pipe.fit(data)

In [17]:
clean_data = cleaner.transform(data)

In [ ]:
clean_data.show()

In [ ]:
clean_data.columns

In [18]:
clean_data = clean_data.select('label', 'features')

In [ ]:
clean_data.show()

In [ ]:
train, test = clean_data.randomSplit([0.7, 0.3])

In [ ]:
spam_detector = nb.fit(train)

In [ ]:
data.printSchema()

In [ ]:
test_results = spam_detector.transform(test)

In [ ]:
test_results.show()

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
acc_eval = MulticlassClassificationEvaluator()

In [ ]:
acc = acc_eval.evaluate(test_results)

In [ ]:
print ('ACC of NB')
print (acc)

In [ ]:
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier

In [ ]:
log_reg = LogisticRegression()

In [ ]:
lg_spam_detector = log_reg.fit(train)

In [ ]:
lg_test_results = lg_spam_detector.transform(test)

In [ ]:
lg_acc = acc_eval.evaluate(lg_test_results)

In [ ]:
lg_acc

In [ ]:
rf = RandomForestClassifier()
gbt = GBTClassifier()

In [ ]:
rf_spam_detector = rf.fit(train)

In [ ]:
gbt_spam_detector = gbt.fit(train)